In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

path_order = "orders.csv"
path_prod = "products.csv"
colnames_order=['OrderID', 'OrderDate', 'ProductID', 'UnitPrice', 'Quantity', 'Discount']
colnames_prod=['ProductID', 'ProductName', 'QuantityPerUnit', 'UnitCost', 'UnitPrice', 'CategoryName']
order = pd.read_csv(path_order, names=colnames_order, header=None)
prod = pd.read_csv(path_prod, names=colnames_prod, header=None)


order

,OrderID,OrderDate,ProductID,UnitPrice,Quantity,Discount
0,OrderID,OrderDate,ProductID,UnitPrice,Quantity,Discount
1,10248,2007-01-30,11,28.62,12,0.0
2,10248,2007-01-30,42,14.04,10,0.0
3,10248,2007-01-30,72,12.71,5,0.0
4,10249,2006-01-29,14,22.8,9,0.0
...,...,...,...,...,...,...
2168,11083,2006-06-16,37,4.5,7,0.0
2169,11083,2006-06-16,42,15.65,10,0.0
2170,11083,2006-06-16,47,25.5,5,0.0
2171,11083,2006-06-16,51,99.9,3,0.0299999993294477


In [19]:
# Для каждой категории продуктов выведите средний доход от продаж. (OrderSum)


order = order.assign(OrderSum = lambda x:'')
order['OrderSum'][0] = 'OrderSum'

for i in range(1, len(order['OrderSum'])):
    order['OrderSum'][i] = (float(order['UnitPrice'][i]) * float(order['Quantity'][i]) * (1 - float(order['Discount'][i])))

Category = {}
for i in range(1, len(prod['CategoryName'])):
    Category[prod['CategoryName'][i]] = 0
    
for i in range(1, len(prod['CategoryName'])):
    Category[prod['CategoryName'][i]] =  Category[prod['CategoryName'][i]] + 1
    
Sum_Id = {}

for i in range(1, len(order['ProductID'])):
    Sum_Id[ order['ProductID'][i]] =  0
    
for i in range(1, len(order['ProductID'])):
    Sum_Id[order['ProductID'][i]] =  float(Sum_Id[order['ProductID'][i]]) + float(order['OrderSum'][i])

Category_sum = {}

for i in range(1, len(prod['CategoryName'])):
    Category_sum[ prod['CategoryName'][i]] = 0
    
for i in range(1, len(prod['CategoryName'])):
    Category_sum[prod['CategoryName'][i]] = Category_sum[prod['CategoryName'][i]] + Sum_Id[prod['ProductID'][i]]

    
Category_average_value = {k: Category_sum[k] / Category[k] for k in Category if k in Category_sum}

print("Средний доход от продаж каждой категории: ")
print(Category_average_value)

Средний доход от продаж каждой категории: 
{'Фрукты': 14450.013539450663, 'Овощи': 38166.98442107003, 'Крупы': 19635.47098781773, 'Морепродукты': 30619.400331876506, 'Приправы': 10720.97545958876, 'Молочные продукты': 4650.5459947858135, 'Кондитерские изделия': 14844.213724807676, 'Мясо/Птица': 19105.729628491965}


In [20]:
#Введите столбец "Profit" - разность между доходами и приблизительной закупочной стоимостью проданных товаров.

Purchase_price_Id = {}

for i in range(1, len(prod['ProductID'])):
    Purchase_price_Id[ prod['ProductID'][i]] =  0
    
for i in range(1, len(prod['ProductID'])):
    Purchase_price_Id[prod['ProductID'][i]] = float(prod['UnitCost'][i])

Purchase_price_Sum = {}

for i in range(1, len(order['ProductID'])):
    Purchase_price_Sum[ order['ProductID'][i]] =  0
    
for i in range(1, len(order['ProductID'])):
    Purchase_price_Sum[order['ProductID'][i]] = Purchase_price_Sum[order['ProductID'][i]] + (Purchase_price_Id[order['ProductID'][i]] * float(order['Quantity'][i]))

Profit_values = {k: Sum_Id[k] - Purchase_price_Sum[k] for k in Purchase_price_Sum if k in Sum_Id}

prod = prod.assign(Profit = lambda x:'')
prod['Profit'][0] = 'Profit' 

for i in range(1, len(prod['Profit'])):
    prod['Profit'][i] = Profit_values[prod['ProductID'][i]]
    
prod

,ProductID,ProductName,QuantityPerUnit,UnitCost,UnitPrice,CategoryName,Profit
0,ProductID,ProductName,QuantityPerUnit,UnitCost,UnitPrice,CategoryName,Profit
1,1,Банан,10,15.37,20.0,Фрукты,2174.28
2,2,Апельсин,10,16.17,19.0,Фрукты,691.086
3,5,Яблоко,100,4.72,6.0,Фрукты,243.313
4,11,Киви,25,21.1,28.0,Фрукты,3344.08
...,...,...,...,...,...,...,...
73,42,Утка,1,10.95,14.0,Мясо/Птица,1430.02
74,52,Говядина,14,70.81,80.0,Мясо/Птица,2848.21
75,56,Свинина,15,34.17,38.0,Мясо/Птица,3423.1
76,57,Фарш свиной,20,16.14,19.5,Мясо/Птица,1185.58


In [24]:
# Найдите по убыванию суммарной прибыли категории товаров, обеспечивающие  её 80 процентов.

Sum_Id = {}
Purchase_price_Sum = {}

for i in range(1, len(order['ProductID'])):
    Sum_Id[ order['ProductID'][i]] =  0
    Purchase_price_Sum[ order['ProductID'][i]] =  0
    
for i in range(1, len(order['ProductID'])):
    OrderDate = order['OrderDate'][i]
    Sum_Id[order['ProductID'][i]] =  float(Sum_Id[order['ProductID'][i]]) + float(order['OrderSum'][i])
    Purchase_price_Sum[order['ProductID'][i]] = Purchase_price_Sum[order['ProductID'][i]] + (Purchase_price_Id[order['ProductID'][i]] * float(order['Quantity'][i]))

Profit = {k: Sum_Id[k] - Purchase_price_Sum[k] for k in Purchase_price_Sum if k in Sum_Id}

Category_Profit = {}

for i in range(1, len(prod['CategoryName'])):
    Category_Profit[ prod['CategoryName'][i]] = 0
    
for i in range(1, len(prod['CategoryName'])):
    Category_Profit[prod['CategoryName'][i]] = Category_Profit[prod['CategoryName'][i]] + Profit[prod['ProductID'][i]]

Category_Profit_sorted = sorted(Category_Profit.values())

Profit_sum = 0

for i in range(1, len(Profit)):
    Profit_sum = Profit_sum + Profit[prod['ProductID'][i]]

sorted_sum = 0

print('Суммарная прибыль категорий товаров, обеспечивающие  её 80 процентов, отсортирована по убыванию:')
for i in range(len(Category_Profit_sorted)-1, len(Category_Profit_sorted)-9, -1):
    print(f"{'Категория: ' + list(Category_Profit.keys())[list(Category_Profit.values()).index(Category_Profit_sorted[i])]:<35} Прибыль за категорию: {Category_Profit_sorted[i]}")
    sorted_sum += Category_Profit_sorted[i]
    if(sorted_sum >= Profit_sum*0.8):
        break

Суммарная прибыль категорий товаров, обеспечивающие  её 80 процентов, отсортирована по убыванию:
Категория: Овощи                    Прибыль за категорию: 93505.46073712053
Категория: Кондитерские изделия     Прибыль за категорию: 28689.84332173053
Категория: Морепродукты             Прибыль за категорию: 24374.982323135588
Категория: Мясо/Птица               Прибыль за категорию: 12379.267399443746


In [26]:
# Выведите аналогичные данные только за 2005 и 2006 год.

Sum_Id = {}
Purchase_price_Sum = {}

for i in range(1, len(order['ProductID'])):
    Sum_Id[ order['ProductID'][i]] =  0
    Purchase_price_Sum[ order['ProductID'][i]] =  0
    
for i in range(1, len(order['ProductID'])):
    OrderDate = order['OrderDate'][i]
    if(OrderDate[:4] == '2005' or OrderDate[:4] == '2006'):
        Sum_Id[order['ProductID'][i]] =  float(Sum_Id[order['ProductID'][i]]) + float(order['OrderSum'][i])
        Purchase_price_Sum[order['ProductID'][i]] = Purchase_price_Sum[order['ProductID'][i]] + (Purchase_price_Id[order['ProductID'][i]] * float(order['Quantity'][i]))

Profit = {k: Sum_Id[k] - Purchase_price_Sum[k] for k in Purchase_price_Sum if k in Sum_Id}

Category_Profit = {}

for i in range(1, len(prod['CategoryName'])):
    Category_Profit[ prod['CategoryName'][i]] = 0
    
for i in range(1, len(prod['CategoryName'])):
    Category_Profit[prod['CategoryName'][i]] = Category_Profit[prod['CategoryName'][i]] + Profit[prod['ProductID'][i]]

Category_Profit_sorted = sorted(Category_Profit.values())

Profit_sum = 0

for i in range(1, len(Profit)):
    Profit_sum = Profit_sum + Profit[prod['ProductID'][i]]

sorted_sum = 0

print('Суммарная прибыль категорий товаров за 2005-2006 год, обеспечивающие  её 80 процентов, отсортирована по убыванию:')
for i in range(len(Category_Profit_sorted)-1, len(Category_Profit_sorted)-9, -1):
    print(f"{'Категория: ' + list(Category_Profit.keys())[list(Category_Profit.values()).index(Category_Profit_sorted[i])]:<35} Прибыль за категорию: {Category_Profit_sorted[i]}")
    sorted_sum += Category_Profit_sorted[i]
    if(sorted_sum >= Profit_sum*0.8):
        break

Суммарная прибыль категорий товаров, обеспечивающие  её 80 процентов, отсортирована по убыванию:
Категория: Овощи                    Прибыль за категорию: 70066.1484911188
Категория: Кондитерские изделия     Прибыль за категорию: 17404.152302461225
Категория: Морепродукты             Прибыль за категорию: 16761.273903283618
Категория: Приправы                 Прибыль за категорию: 8875.249925775903
